In [ ]:
!pip install xgboost optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.3/212.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 kB 6.5 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import logging
import os
import random
import warnings

import lightgbm as lgb
import numpy as np
import optuna
import pandas as pd

from optuna import Trial, visualization
from optuna.samplers import TPESampler

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier


warnings.filterwarnings("ignore")
seed = 524

# Data load

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/미니프로젝트5차_3_5일차 실습자료/train_dataset.csv")
test_kaggle = pd.read_csv("/content/drive/MyDrive/미니프로젝트5차_3_5일차 실습자료/test_dataset_real.csv")

label_dic = {
    "WALKING": 0,
    "WALKING_UPSTAIRS": 1,
    "WALKING_DOWNSTAIRS": 2,
    "LAYING": 3,
    "STANDING": 4,
    "SITTING": 5
}
X, y = train.drop(["Activity_dynamic", 'Activity'], axis=1), train["Activity_dynamic"]

In [ ]:
import numpy as np
from joblib import Parallel, delayed
from scipy.interpolate import interp1d
from scipy.special import erf, erfinv
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils.validation import FLOAT_DTYPES, check_array, check_is_fitted


class GaussRankScaler(BaseEstimator, TransformerMixin):

    def __init__(
        self,
        epsilon=1e-4,
        copy=True,
        n_jobs=None,
        interp_kind="linear",
        interp_copy=False,
    ):
        self.epsilon = epsilon
        self.copy = copy
        self.interp_kind = interp_kind
        self.interp_copy = interp_copy
        self.fill_value = "extrapolate"
        self.n_jobs = n_jobs
        self.bound = 1.0 - self.epsilon

    def fit(self, X, y=None):

        X = check_array(
            X, copy=self.copy, estimator=self, dtype=FLOAT_DTYPES, force_all_finite=True
        )

        self.interp_func_ = Parallel(n_jobs=self.n_jobs)(
            delayed(self._fit)(x) for x in X.T
        )
        return self

    def _fit(self, x):
        x = self.drop_duplicates(x)
        rank = np.argsort(np.argsort(x))
        factor = np.max(rank) / 2.0 * self.bound
        scaled_rank = np.clip(rank / factor - self.bound, -self.bound, self.bound)
        return interp1d(
            x,
            scaled_rank,
            kind=self.interp_kind,
            copy=self.interp_copy,
            fill_value=self.fill_value,
        )

    def transform(self, X, copy=None):

        check_is_fitted(self, "interp_func_")

        copy = copy if copy is not None else self.copy
        X = check_array(
            X, copy=copy, estimator=self, dtype=FLOAT_DTYPES, force_all_finite=True
        )

        X = np.array(
            Parallel(n_jobs=self.n_jobs)(
                delayed(self._transform)(i, x) for i, x in enumerate(X.T)
            )
        ).T
        return X

    def _transform(self, i, x):
        clipped = np.clip(self.interp_func_[i](x), -self.bound, self.bound)
        return erfinv(clipped)

    def inverse_transform(self, X, copy=None):
        check_is_fitted(self, "interp_func_")

        copy = copy if copy is not None else self.copy
        X = check_array(
            X, copy=copy, estimator=self, dtype=FLOAT_DTYPES, force_all_finite=True
        )

        X = np.array(
            Parallel(n_jobs=self.n_jobs)(
                delayed(self._inverse_transform)(i, x) for i, x in enumerate(X.T)
            )
        ).T
        return X

    def _inverse_transform(self, i, x):
        inv_interp_func = interp1d(
            self.interp_func_[i].y,
            self.interp_func_[i].x,
            kind=self.interp_kind,
            copy=self.interp_copy,
            fill_value=self.fill_value,
        )
        return inv_interp_func(erf(x))

    @staticmethod
    def drop_duplicates(x):
        is_unique = np.zeros_like(x, dtype=bool)
        is_unique[np.unique(x, return_index=True)[1]] = True
        return x[is_unique]

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_kaggle_scaled = scaler.transform(test_kaggle)

### fit, predict 할 때 X, y 항상 잘 확인하기
## optuna로 하이퍼파라미터 서칭

In [ ]:
class MODEL(object):
    def __init__(self, X, y, cv):
        self.X = X
        self.y = y
        self.cv = cv

    def __call__(self, trial):
        max_depth = trial.suggest_int("max_depth", 1, 12) # 20
        max_leaves = trial.suggest_int('max_leaves', max_depth,  2**max_depth - 1)

        xgb_param = { #default 는 지정안함
            # 'booster' : 'dart',#추가
            'objective' : 'multi:softmax',
            'eval_metric' : 'merror',
            'seed' : seed,
            'n_estimators': trial.suggest_int('n_estimators', 100, 3000),
            'learning_rate': trial.suggest_float("learning_rate", 0.0, 0.4),
            'eta' : trial.suggest_float('eta', 0.0, 0.3),
            'gamma' : trial.suggest_int('gamma', 0, 4),
            'min_child_weight' : trial.suggest_int('min_child_weight', 0, 5),
            'max_delta_step' : trial.suggest_int('max_delta_step', 1, 10),
            'subsample' : trial.suggest_float('subsample', 0.7, 1.0), #일반적으로 >=0.5
            # 'sampling_method' : 'sumsample',
            'colsample_bytree' : trial.suggest_float('colsample_bytree',0.7, 1.0),
            #'colsample_bylevel' : trial.suggest_float('colsample_bylevel',0.0, 0.3),
            #'colsample_bynode' : trial.suggest_float('colsample_bynode',0.0, 0.3), #계속 분할시키는 것 colsample는 누적작동
            # 'reg_lambda' : 1, #l2규제 해줘야할지 판단
        }

        model_xgb = XGBClassifier(**xgb_param, max_depth=max_depth)#, max_leaves=max_leaves)

        if self.cv:
            cv = StratifiedKFold(n_splits=self.cv, shuffle=True, random_state=seed)
            score = cross_val_score(model_xgb, self.X, self.y, cv=cv, scoring="f1").mean()

        else:
            self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X, self.y, test_size=0.2, random_state=seed, stratify=self.y)
            score = f1_score(self.y_test,model_xgb.fit(self.X_train, self.y_train).predict(self.X_test),average="macro",)
        return score

XGB_ob = MODEL(X_scaled, y, cv=0)
sampler = TPESampler(seed=seed)
XGB = optuna.create_study(
    study_name="xgb_optuna",
    direction="maximize",
    pruner=optuna.pruners.HyperbandPruner(max_resource="auto"),
    sampler=sampler,
)

XGB.optimize(XGB_ob, n_trials=10)

In [ ]:
optuna.visualization.plot_parallel_coordinate(XGB)

In [ ]:
optuna.visualization.plot_param_importances(XGB)

### real modeling

In [ ]:
# 파라미터 확인
XGB.best_params

{'max_depth': 7,
 'n_estimators': 1818,
 'learning_rate': 0.01858016508799909,
 'eta': 0.1822634555704315,
 'gamma': 0,
 'min_child_weight': 0,
 'max_delta_step': 10,
 'subsample': 0.9896896099223678,
 'colsample_bytree': 0.9425192044349383}

In [ ]:
best_params = XGB.best_params

In [ ]:
bestparams =  {'max_depth': 3, 'max_leaves': 7, 'n_estimators': 2514, 'learning_rate': 0.08493564427131046, 'eta': 0.05454749016213018, 'gamma': 0, 'min_child_weight': 1, 'max_delta_step': 6, 'subsample': 0.8295835055926347, 'colsample_bytree': 0.7873687420594125}

In [ ]:
model = XGBClassifier(**best_params)

In [ ]:
model = XGBClassifier(**bestparams)

In [ ]:
model = model.fit(X_scaled,y)
pred = model.predict(test_kaggle_scaled)

## 앙상블

In [ ]:
from sklearn.ensemble import VotingClassifier
bestparams =  {'max_depth': 3, 'max_leaves': 7, 'n_estimators': 2514, 'learning_rate': 0.08493564427131046, 'eta': 0.05454749016213018, 'gamma': 0, 'min_child_weight': 1, 'max_delta_step': 6, 'subsample': 0.8295835055926347, 'colsample_bytree': 0.7873687420594125}
cl1 = XGBClassifier(**bestparams, seed = random.randint(0,10000))
cl2 = XGBClassifier(**bestparams, seed = random.randint(0,10000))
cl3 = XGBClassifier(**bestparams, seed = random.randint(0,10000))
vote = VotingClassifier(
    estimators = [('xgb1', cl1), ('xgb2', cl2), ('xgb3', cl3)],
    voting = 'soft'
)
vote.fit(X_scaled, y)
proba_pred = vote.predict_proba(test_kaggle_scaled)
pred = np.argmax(proba_pred, axis=1)

In [ ]:
sub = pd.read_csv('/content/drive/MyDrive/미니프로젝트5차_3_5일차 실습자료/sample_submission.csv')
sub

,ID,Activity
0,0,5
1,1,4
2,2,0
3,3,5
4,4,3
...,...,...
1466,1466,5
1467,1467,5
1468,1468,2
1469,1469,3


In [ ]:
sub['Activity'] = pred
sub.to_csv('test2.csv', index=False)
sub

,ID,Activity
0,0,5
1,1,4
2,2,0
3,3,5
4,4,4
...,...,...
1466,1466,3
1467,1467,4
1468,1468,0
1469,1469,3
